In [1]:
%display latex

In [2]:
version()

/tmp/ipykernel_2137247/2131152398.py:1: DeprecationWarning: Use sage.version instead.
See https://github.com/sagemath/sage/issues/39015 for details.
  version()


'SageMath version 10.7, Release Date: 2025-08-09'

In [3]:
M = Manifold(4, r'\mathcal{M}')
M

4-dimensional differentiable manifold \mathcal{M}

In [11]:
X.<t,z,th,ph> = M.chart(r"t z th:(0,pi):\theta ph:(0,2*pi):\phi")
X

Chart (\mathcal{M}, (t, z, th, ph))

In [12]:
Lambda = var("Lambda")

In [15]:
g = M.metric("g")
g[0,0] = -1
g[1,1] = e^(2*t*(Lambda^0.5))
g[2,2] = 1 / Lambda
g[3,3] = sin(th)^2 / Lambda
g[:]

[                  -1                    0                    0                    0]
[                   0 e^(2*sqrt(Lambda)*t)                    0                    0]
[                   0                    0             1/Lambda                    0]
[                   0                    0                    0     sin(th)^2/Lambda]

In [17]:
xi = M.vector_field(r'\xi')
xi[0] = function('xi_0')(t, z, th, ph)
xi[1] = function('xi_1')(t, z, th, ph)
xi[2] = function('xi_2')(t, z, th, ph)
xi[3] = function('xi_3')(t, z, th, ph)
xi.display()

\xi = xi_0(t, z, th, ph) ∂/∂t + xi_1(t, z, th, ph) ∂/∂z + xi_2(t, z, th, ph) ∂/∂th + xi_3(t, z, th, ph) ∂/∂ph

In [20]:
nabla = g.connection()
killing_eqs = nabla(xi.down(g)).symmetrize()
killing_eqs.display_comp(only_nonredundant=True)

X_t,t = -d(xi_0)/dt 
X_t,z = 1/2*e^(2*sqrt(Lambda)*t)*d(xi_1)/dt - 1/2*d(xi_0)/dz 
X_t,th = -1/2*(Lambda*d(xi_0)/dth - d(xi_2)/dt)/Lambda 
X_t,ph = 1/2*(sin(th)^2*d(xi_3)/dt - Lambda*d(xi_0)/dph)/Lambda 
X_z,z = sqrt(Lambda)*e^(2*sqrt(Lambda)*t)*xi_0(t, z, th, ph) + e^(2*sqrt(Lambda)*t)*d(xi_1)/dz 
X_z,th = 1/2*(Lambda*e^(2*sqrt(Lambda)*t)*d(xi_1)/dth + d(xi_2)/dz)/Lambda 
X_z,ph = 1/2*(Lambda*e^(2*sqrt(Lambda)*t)*d(xi_1)/dph + sin(th)^2*d(xi_3)/dz)/Lambda 
X_th,th = d(xi_2)/dth/Lambda 
X_th,ph = 1/2*(sin(th)^2*d(xi_3)/dth + d(xi_2)/dph)/Lambda 
X_ph,ph = (cos(th)*sin(th)*xi_2(t, z, th, ph) + sin(th)^2*d(xi_3)/dph)/Lambda

In [22]:
from sage.all import *
from sage.manifolds.all import Manifold

def run_killing_vector_analysis():
    # 1. Setup the Manifold
    # ---------------------
    M = Manifold(4, 'M', structure='Lorentzian')
    X.<t,z,th,ph> = M.chart(r"t z th:\theta ph:\phi")
    
    # Define Lambda and H = sqrt(Lambda)
    Lam = var('Lam', domain='positive')
    H = sqrt(Lam)
    
    # 2. Define the Metric
    # --------------------
    g = M.metric()
    # ds^2 = -dt^2 + e^(2*H*t) dz^2 + (1/Lam)(dtheta^2 + sin^2 theta dphi^2)
    g[0,0] = -1
    g[1,1] = exp(2 * H * t)
    g[2,2] = 1/Lam
    g[3,3] = (1/Lam) * sin(th)^2
    
    print("Metric defined:")
    print(g.display())
    print("-" * 40)

    # 3. Display the General Killing Equations
    # ----------------------------------------
    print("General Killing Equations for this metric:")
    # We create a generic vector field xi to show the PDEs
    xi = M.vector_field(name='xi')
    # Define undefined functions for components
    f_t = function('f_t')(t, z, th, ph)
    f_z = function('f_z')(t, z, th, ph)
    f_th = function('f_th')(t, z, th, ph)
    f_ph = function('f_ph')(t, z, th, ph)
    xi[:] = [f_t, f_z, f_th, f_ph]
    
    Lie_g = g.lie_derivative(xi)
    
    # Print only non-zero equations
    count = 1
    for i in range(4):
        for j in range(i, 4):
            eqn = Lie_g[i,j].expr()
            if eqn != 0:
                print(f"Eq {count} ({X[i]},{X[j]}): {eqn} == 0")
                count += 1
    print("-" * 40)

    # 4. Define and Verify the 6 Solutions
    # ------------------------------------
    print("Verifying the 6 Killing Vectors for dS2 x S2 geometry:")
    
    killing_vectors = []

    # --- Spherical Symmetry (SO(3)) ---
    # V1: Rotation around z
    v1 = M.vector_field(name='K1')
    v1[:] = [0, 0, 0, 1]
    killing_vectors.append(v1)

    # V2: Rotation x/y 1
    v2 = M.vector_field(name='K2')
    v2[:] = [0, 0, sin(ph), cos(ph)*cot(th)]
    killing_vectors.append(v2)

    # V3: Rotation x/y 2
    v3 = M.vector_field(name='K3')
    v3[:] = [0, 0, cos(ph), -sin(ph)*cot(th)]
    killing_vectors.append(v3)

    # --- de Sitter Symmetry (SO(1,2)) ---
    # V4: Spatial translation z
    v4 = M.vector_field(name='K4')
    v4[:] = [0, 1, 0, 0]
    killing_vectors.append(v4)

    # V5: Dilation / Boost
    v5 = M.vector_field(name='K5')
    v5[:] = [1, -H*z, 0, 0]
    killing_vectors.append(v5)

    # V6: Special Conformal Transformation
    v6 = M.vector_field(name='K6')
    term_z = -(H/2)*z^2 - (1/(2*H))*exp(-2*H*t)
    v6[:] = [z, term_z, 0, 0]
    killing_vectors.append(v6)

    # --- Verification Loop ---
    for i, vec in enumerate(killing_vectors):
        # Calculate Lie Derivative
        lg = g.lie_derivative(vec)
        
        # Check if all components are zero
        is_killing = True
        for a in range(4):
            for b in range(4):
                if lg[a,b].expr().simplify_full() != 0:
                    is_killing = False
        
        status = "[PASS]" if is_killing else "[FAIL]"
        print(f"{status} Vector {i+1}: {vec.display()}")

if __name__ == "__main__":
    run_killing_vector_analysis()

Metric defined:
g = -dt⊗dt + e^(2*sqrt(Lam)*t) dz⊗dz + 1/Lam dth⊗dth + sin(th)^2/Lam dph⊗dph
----------------------------------------
General Killing Equations for this metric:
----------------------------------------
Verifying the 6 Killing Vectors for dS2 x S2 geometry:
[PASS] Vector 1: K1 = ∂/∂ph
[PASS] Vector 2: K2 = sin(ph) ∂/∂th + cos(ph)*cot(th) ∂/∂ph
[PASS] Vector 3: K3 = cos(ph) ∂/∂th - cot(th)*sin(ph) ∂/∂ph
[PASS] Vector 4: K4 = ∂/∂z
[PASS] Vector 5: K5 = ∂/∂t - sqrt(Lam)*z ∂/∂z
[PASS] Vector 6: K6 = z ∂/∂t + (-1/2*sqrt(Lam)*z^2 - 1/2*e^(-2*sqrt(Lam)*t)/sqrt(Lam)) ∂/∂z
